In [1]:
import numpy as np
import csv
import matplotlib.pyplot as plt
import math
import random
import pandas as pd

import seaborn as sns
from __future__ import division
import time
from cvxpy import *

from distutils.log import error
import errno
import numpy as np
import pandas as pd
import time
from datetime import date
import cvxpy as cp

import parameters

In [2]:
User_PV_Data = pd.read_csv('./Solar_Demand.csv')

# get_all_values gives a list of rows.


userdata = np.array(User_PV_Data.values)


print(User_PV_Data)

     Time  PV_Output  PV_Percentage  Demand_kW  Grid_Cost  Unnamed: 5  \
0    0:00        0.0            0.0       0.12       0.01         NaN   
1    0:15        0.0            0.0       0.12       0.01         NaN   
2    0:30        0.0            0.0       0.12       0.01         NaN   
3    0:45        0.0            0.0       0.12       0.01         NaN   
4    1:00        0.0            0.0       0.16       0.01         NaN   
..    ...        ...            ...        ...        ...         ...   
91  22:45        0.0            0.0       0.20       0.01         NaN   
92  23:00        0.0            0.0       0.16       0.01         NaN   
93  23:15        0.0            0.0       0.16       0.01         NaN   
94  23:30        0.0            0.0       0.16       0.01         NaN   
95  23:45        0.0            0.0       0.16       0.01         NaN   

    Unnamed: 6  
0          NaN  
1          NaN  
2          NaN  
3          NaN  
4          NaN  
..         ...  
91  

In [3]:
prof=pd.Series(dtype='object')

prof.loc['Solar']=User_PV_Data.iloc[:, 2]
prof.loc['Load']=User_PV_Data.iloc[:, 3]
prof.loc['Price']=User_PV_Data.iloc[:, 4]*100

k=prof.Load.shape[0]

In [5]:
prof.Load

0     0.12
1     0.12
2     0.12
3     0.12
4     0.16
      ... 
91    0.20
92    0.16
93    0.16
94    0.16
95    0.16
Name: Demand_kW, Length: 96, dtype: float64

In [6]:
import pyomo.environ as pyo

In [7]:
m = pyo.ConcreteModel()

m.Time = pyo.Set(initialize=pyo.RangeSet(k),ordered=True)

#scalar parameters
m.Kpv = pyo.Param(initialize=300) # Solar capacity cost ($/kW)
m.Kb = pyo.Param(initialize= 150) # Battery capacity cost ($/kWh)
m.VCC = pyo.Param(initialize= 75) # Battery discharge cost ($/MWh)
m.nu =pyo.Param(initialize= 0.85) # Battery efficiency
m.r = pyo.Param(initialize= 0.004) # Real interest rate
m.b_min_cap = pyo.Param(initialize= 0) # min battery cap

#time indexed parameters
m.grid_cost = pyo.Param(m.Time,initialize=dict(enumerate(prof.Price,1)),within=pyo.Any)
m.demand = pyo.Param(m.Time,initialize=dict(enumerate(prof.Load,1)),within=pyo.Any)
m.solar = pyo.Param(m.Time,initialize=dict(enumerate(prof.Solar,1)),within=pyo.Any)

In [8]:
#time indexed Variables
m.SOC= pyo.Var(m.Time, domain=pyo.NonNegativeReals) # Battery state of charge at step k (MWh)
m.Bc = pyo.Var(m.Time, domain=pyo.NonNegativeReals) # Battery charge at step k (MWh)
m.Bd = pyo.Var(m.Time, domain=pyo.NonNegativeReals) # Battery dischcharge at step k (MWh)
m.Ge = pyo.Var(m.Time, domain=pyo.NonNegativeReals) # Grid energy export at step k (MWh)
m.Gi = pyo.Var(m.Time, domain=pyo.NonNegativeReals) # Grid energy import at step k (MWh)
m.Dpv = pyo.Var(m.Time, domain=pyo.NonNegativeReals) # Curtailed PV production at step k (MWh)
m.Dl =pyo.Var(m.Time, domain=pyo.NonNegativeReals) # Cutailed load at step k (MWh)
m.X =pyo.Var(m.Time, domain=pyo.NonNegativeReals) # Solar capacity MW
m.Y = pyo.Var(m.Time, domain=pyo.NonNegativeReals) # Battery capacity MWh

In [9]:
#CONSTRAINTS
#Purchase constraint
def purchase_constraint(m,k):
    return m.Gi[k] >= m.demand[k] - m.Bd[k] + m.Bc[k] - X[k]*m.solar[k]

#State of charge constraint
def soc_constraint(m,k):
        return m.SOC[k+1] == m.SOC[k] + m.Bc[k]*m.nu - m.Bd[k]*m.nu
    
#Discharge and charge constraints
def discharge_constraint_1(m,k):
    """ Maximum discharge rate within time set k"""
    return model.Bd[k] <= m.Y[k]/4

def discharge_constraint_2(m,k):
    """ Sets the maximum energy available to be discharged as the SOC - minimum SOC """
    return m.Bd[k] <= m.SOC[k] - m.b_min_cap

def charge_constraint_1(m,k):
    """ Maximum charge rate within a single hour """
    return model.Bc[k] <= m.Y[k]/4 

def charge_constraint_2(m,k):
    """ Sets the maximum energy available to be cahrge as the SOC max """
    return model.bc[t]  <= m.Y[k] - m.SOC[k]



In [10]:
#constraints
m.Gi_c = pyo.Constraint(m.Time, rule = purchase_constraint)
m.SOC_c = pyo.Constraint(m.Time, rule = soc_constraint)
m.Bd_c1 = pyo.Constraint(m.Time, rule = discharge_constraint_1)
m.Bd_c2 = pyo.Constraint(m.Time,rule = discharge_constraint_2)
m.Bc_c1 = pyo.Constraint(m.Time,rule = charge_constraint_1)
m.Bc_c2 = pyo.Constraint(m.Time,rule = charge_constraint_2)

#OBJECTIVE 
expr = sum(m.Gi[k] * m.grid_cost[k]  for k in m.Time)
model.objective = pyo.Objective(rule = expr, sense = pyo.minimize)    
opt = pyo.SolverFactory('cbc',executable='/usr/bin/cbc')
results = opt.solve(model)
results.write()



ERROR: Rule failed when generating expression for Constraint Gi_c with index
    1: NameError: name 'X' is not defined
ERROR: Constructing component 'Gi_c' from data=None failed: NameError: name
    'X' is not defined


NameError: name 'X' is not defined